In [0]:
import dlt
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
 
# =========================================
# DATE_DIM  (Type 1 - Fixed)
# =========================================
# =========================================
# DATE_DIM  (Type 1 - Fixed)
# =========================================
@dlt.table(
    name="DATE_DIM",
    comment="Date dimension (Fixed Type 1) - Power BI Compatible",
    table_properties={"quality": "gold", "dimension_type": "type1"}
)
def date_dim():
    df = spark.read.table("midterm_project.gold.food_inspections_merged")
    date_df = (
        df.select("INSPECTION_DATE")
          .distinct()
          .filter(F.col("INSPECTION_DATE").isNotNull())
          # DATE_SK as INTEGER (yyyyMMdd format)
          .withColumn("DATE_SK", 
                      F.date_format(F.col("INSPECTION_DATE"), "yyyyMMdd").cast("int"))
          # FULL_DATE as proper DATE type (NOT STRING!)
          .withColumn("FULL_DATE", 
                      F.col("INSPECTION_DATE").cast("date"))
          # Day attributes
          .withColumn("DAY_NUM", 
                      F.dayofmonth("INSPECTION_DATE").cast("int"))
          .withColumn("WEEKDAY_ABBR", 
                      F.date_format("INSPECTION_DATE", "E"))
          .withColumn("WEEKDAY_NAME", 
                      F.date_format("INSPECTION_DATE", "EEEE"))
          .withColumn("WEEKDAY_NUM", 
                      F.dayofweek("INSPECTION_DATE").cast("int"))
          .withColumn("DAY_OF_YEAR_NUM", 
                      F.dayofyear("INSPECTION_DATE").cast("int"))
          # Week attributes
          .withColumn("WEEK_OF_YEAR", 
                      F.weekofyear("INSPECTION_DATE").cast("int"))
          # Month attributes
          .withColumn("MONTH_NUM", 
                      F.month("INSPECTION_DATE").cast("int"))
          .withColumn("MONTH_ABBR", 
                      F.date_format("INSPECTION_DATE", "MMM"))
          .withColumn("MONTH_NAME", 
                      F.date_format("INSPECTION_DATE", "MMMM"))
          # Quarter attributes
          .withColumn("QUARTER_NUM", 
                      F.quarter("INSPECTION_DATE").cast("int"))
          .withColumn("QUARTER_NAME", 
                      F.concat(F.lit("Q"), F.quarter("INSPECTION_DATE")))
          # Year attributes
          .withColumn("YEAR_NUM", 
                      F.year("INSPECTION_DATE").cast("int"))
          # Month boundaries as DATE type
          .withColumn("FIRST_DAY_OF_MONTH", 
                      F.trunc("INSPECTION_DATE", "month").cast("date"))
          .withColumn("LAST_DAY_OF_MONTH", 
                      F.last_day("INSPECTION_DATE").cast("date"))
          # Weekend flag
          .withColumn("IS_WEEKEND", 
                      F.when(F.dayofweek("INSPECTION_DATE").isin([1,7]), "Y").otherwise("N"))
          # Year-Month for sorting (as string for display, but also add numeric version)
          .withColumn("YEAR_MONTH", 
                      F.date_format("INSPECTION_DATE", "yyyy-MM"))
          .withColumn("YEAR_MONTH_NUM", 
                      F.date_format("INSPECTION_DATE", "yyyyMM").cast("int"))
          # Remove duplicates
          .dropDuplicates(["DATE_SK"])
          # Select columns in proper order
          .select(
              "DATE_SK",
              "FULL_DATE",
              "DAY_NUM",
              "WEEKDAY_ABBR",
              "WEEKDAY_NAME",
              "WEEKDAY_NUM",
              "DAY_OF_YEAR_NUM",
              "WEEK_OF_YEAR",
              "MONTH_NUM",
              "MONTH_ABBR",
              "MONTH_NAME",
              "QUARTER_NUM",
              "QUARTER_NAME",
              "YEAR_NUM",
              "YEAR_MONTH",
              "YEAR_MONTH_NUM",
              "FIRST_DAY_OF_MONTH",
              "LAST_DAY_OF_MONTH",
              "IS_WEEKEND"
          )
    )
    return date_df
 
# =========================================
# LOCATION_DIM  (Type 1)
# =========================================
@dlt.table(
    name="LOCATION_DIM",
    comment="Location dimension (Type 1)",
    table_properties={"quality": "gold", "dimension_type": "type1"}
)
def location_dim():
    df = spark.read.table("midterm_project.gold.food_inspections_merged")
    loc_df = (
        df.select("STREET_ADDRESS", "CITY", "STATE", "ZIP_CODE", "LATITUDE", "LONGITUDE")
          .filter(F.col("STREET_ADDRESS").isNotNull())
          .dropDuplicates(["STREET_ADDRESS", "CITY", "STATE", "ZIP_CODE"])
          .withColumn("LOCATION_SK", F.monotonically_increasing_id().cast("decimal(10,0)"))
          .withColumn("DI_LOAD_DT", F.current_date())
          .withColumn("DI_JOB_ID", F.lit("GOLD_LOAD_001"))
    )
    return loc_df
 
# =========================================
# BUSINESS_DIM  (Type 2 - MANUAL SCD)
# =========================================
# =========================================

@dlt.table(

    name="BUSINESS_DIM",

    comment="Business dimension with MANUAL SCD Type 2 implementation",

    table_properties={"quality": "gold", "dimension_type": "type2"}

)

def business_dim():

    """

    Manual SCD Type 2: Track historical changes to business attributes.

    Creates new version when BUSINESS_NAME, AKA_NAME, or BUSINESS_TYPE changes.

    """

    df = spark.read.table("midterm_project.gold.food_inspections_merged")

    # Step 1: Get all unique combinations of business attributes with their first appearance date

    business_changes = (

        df.select(

            "LICENSE_NUMBER",

            "BUSINESS_NAME",

            "AKA_NAME",

            F.col("FACILITY_TYPE").alias("BUSINESS_TYPE"),  # Alias here

            "INSPECTION_DATE"

        )

        .filter(F.col("LICENSE_NUMBER").isNotNull())

        .dropDuplicates(["LICENSE_NUMBER", "BUSINESS_NAME", "AKA_NAME", "BUSINESS_TYPE"])  # ← FIXED: Use BUSINESS_TYPE

    )

    # Step 2: For each unique combination, get the earliest date (when this version started)

    business_versions = (

        business_changes

        .groupBy("LICENSE_NUMBER", "BUSINESS_NAME", "AKA_NAME", "BUSINESS_TYPE")

        .agg(F.min("INSPECTION_DATE").alias("EFFECTIVE_START_DATE"))

    )

    # Step 3: Add window to determine end dates

    window_spec = Window.partitionBy("LICENSE_NUMBER").orderBy("EFFECTIVE_START_DATE")

    business_with_end_dates = (

        business_versions

        .withColumn("NEXT_START_DATE", F.lead("EFFECTIVE_START_DATE").over(window_spec))

        .withColumn(

            "EFFECTIVE_END_DATE", 

            F.when(F.col("NEXT_START_DATE").isNotNull(), 

                   F.date_sub(F.col("NEXT_START_DATE"), 1))

            .otherwise(F.lit(None))

        )

        .drop("NEXT_START_DATE")

    )

    # Step 4: Add IS_CURRENT flag and surrogate key

    business_scd2 = (

        business_with_end_dates

        .withColumn(

            "IS_CURRENT",

            F.when(F.col("EFFECTIVE_END_DATE").isNull(), "Y").otherwise("N")

        )

        .withColumn(

            "VERSION_NUMBER",

            F.row_number().over(Window.partitionBy("LICENSE_NUMBER").orderBy("EFFECTIVE_START_DATE"))

        )

        .withColumn(

            "BUSINESS_SK",

            # Create a unique numeric key: LICENSE_NUMBER * 1000 + VERSION_NUMBER

            (F.col("LICENSE_NUMBER") * 1000 + F.col("VERSION_NUMBER")).cast("decimal(20,0)")

        )

        .withColumn("DI_JOB_ID", F.lit("GOLD_LOAD_001"))

        .withColumn("DI_LOAD_DT", F.current_date())

        .select(

            "BUSINESS_SK",

            "LICENSE_NUMBER",

            "BUSINESS_NAME",

            "AKA_NAME",

            "BUSINESS_TYPE",

            "EFFECTIVE_START_DATE",

            "EFFECTIVE_END_DATE",

            "IS_CURRENT",

            "VERSION_NUMBER",

            "DI_JOB_ID",

            "DI_LOAD_DT"

        )

    )

    return business_scd2
  
# =========================================
# VIOLATION_DIM  (Type 1)
# =========================================
@dlt.table(
    name="VIOLATION_DIM",
    comment="Violation dimension (Type 1)",
    table_properties={"quality": "gold", "dimension_type": "type1"}
)
def violation_dim():
    df = spark.read.table("midterm_project.gold.food_inspections_merged")
    viol_df = (
        df.select("VIOLATION_CODE", "VIOLATION_DESCRIPTION")
          .filter(F.col("VIOLATION_CODE").isNotNull())
          .dropDuplicates(["VIOLATION_CODE"])
          .withColumn("VIOLATION_SK", F.monotonically_increasing_id().cast("decimal(10,0)"))
          .withColumn("DI_LOAD_DT", F.current_date())
          .withColumn("DI_JOB_ID", F.lit("GOLD_LOAD_001"))
    )
    return viol_df
 
# =========================================
# FACT_FOOD_INSP  (Atomic Fact Table)
# =========================================
@dlt.table(
    name="FACT_FOOD_INSP",
    comment="Fact table - one record per inspection × violation",
    table_properties={"quality": "gold", "fact_type": "atomic"}
)
def fact_food_insp():
    """
    Fact table that links to the CURRENT version of each business at time of inspection.
    Uses inspection date to determine which business version was active.
    """
    df = spark.read.table("midterm_project.gold.food_inspections_merged")
    # Read Dimension Tables
    date_dim = dlt.read("DATE_DIM").select("DATE_SK", "FULL_DATE")
    loc_dim = dlt.read("LOCATION_DIM").select("LOCATION_SK", "STREET_ADDRESS", "CITY", "STATE", "ZIP_CODE")
    viol_dim = dlt.read("VIOLATION_DIM").select("VIOLATION_SK", "VIOLATION_CODE")
    biz_dim = dlt.read("BUSINESS_DIM").select(
        "BUSINESS_SK", 
        "LICENSE_NUMBER", 
        "EFFECTIVE_START_DATE", 
        "EFFECTIVE_END_DATE",
        "IS_CURRENT"
    )
    # Join with dimensions
    # For BUSINESS_DIM: Join based on which version was active at inspection time
    fact_df = (
        df
        # Join DATE_DIM
        .join(date_dim, df.INSPECTION_DATE == date_dim.FULL_DATE, "left")
        # Join LOCATION_DIM
        .join(loc_dim,
              (df.STREET_ADDRESS == loc_dim.STREET_ADDRESS) &
              (df.ZIP_CODE == loc_dim.ZIP_CODE), "left")
        # Join VIOLATION_DIM
        .join(viol_dim, df.VIOLATION_CODE == viol_dim.VIOLATION_CODE, "left")
        # Join BUSINESS_DIM - get the version that was active at inspection time
        .join(
            biz_dim,
            (df.LICENSE_NUMBER == biz_dim.LICENSE_NUMBER) &
            (df.INSPECTION_DATE >= biz_dim.EFFECTIVE_START_DATE) &
            (
                (df.INSPECTION_DATE <= biz_dim.EFFECTIVE_END_DATE) | 
                (biz_dim.EFFECTIVE_END_DATE.isNull())
            ),
            "left"
        )
        # Create fact columns
        .withColumn("INSPECTION_SK", F.monotonically_increasing_id().cast("decimal(10,0)"))
        .withColumn("DI_JOB_ID", F.lit("GOLD_LOAD_001"))
        .withColumn("DI_LOAD_DT", F.current_date())
        .select(
            "INSPECTION_SK",
            "LOCATION_SK",
            "VIOLATION_SK",
            "BUSINESS_SK",
            "DATE_SK",
            F.col("INSPECTION_ID").cast("decimal(30,0)"),
            "INSPECTION_TYPE",
            "INSPECTION_RESULT",
            "RISK_LEVEL",
            F.col("INSPECTION_SCORE").cast("decimal(10,0)"),
            "DI_JOB_ID",
            "DI_LOAD_DT"
        )
        .dropDuplicates(["INSPECTION_ID", "VIOLATION_SK"])
    )
    return fact_df